<a href="https://colab.research.google.com/github/ferdinandrafols/IA_LLMs/blob/main/gsi073_aula0_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparação dos dados

Esta tarefa é inverter sequências de caracteres. Exemplo: **aabcd** em **dcbaa**.


In [ ]:
import torch
import torch.nn as nn
import random

# ===== 1. Dicionário e funções básicas =====
PAD_TOKEN = ' ' # Token para preenchimento
SOS_TOKEN = '<S>' # Token para "Start Of Sequence"

chars = list("abcde") + [PAD_TOKEN, SOS_TOKEN] # Define o conjunto de caracteres permitidos e tokens especiais
vocab = {ch: i for i, ch in enumerate(chars)}  # Mapeia cada caractere para um índice numérico
inv_vocab = {i: ch for ch, i in vocab.items()} # Cria um dicionário inverso para decodificar índices para caracteres
vocab_size = len(vocab)  # Quantidade total de tokens possíveis

def encode(s):  # Converte uma string em uma sequência de índices numéricos
    return torch.tensor([vocab[c] for c in s], dtype=torch.long)

def decode(t):  # Converte uma sequência de índices numéricos de volta para string
    return ''.join(inv_vocab[int(x)] for x in t if inv_vocab[int(x)] not in [PAD_TOKEN, SOS_TOKEN]) # Ignora PAD/SOS na decodificação final

def random_seq(n=5):  # Gera uma sequência aleatória de tamanho n usando apenas 'abcde'
    return ''.join(random.choice(chars[:5]) for _ in range(n))  # Usa apenas a-e

# ===== 2. Gerar dados =====
max_len = 5 # Definir max_len fixo para este problema

def pad_input(x):  # Preenche a sequência de entrada com PAD_TOKEN
    return torch.cat([x, torch.tensor([vocab[PAD_TOKEN]] * (max_len - len(x)))], dim=0)

def pad_target(y): # Preenche a sequência alvo com SOS_TOKEN no início e PAD_TOKEN no final, se necessário
    # A sequência alvo terá SOS_TOKEN + sequência invertida + padding (se max_len_target > len(seq)+1)
    target_with_sos = torch.cat([torch.tensor([vocab[SOS_TOKEN]], dtype=torch.long), y], dim=0)
    # A sequência de saída do decoder é (len da seq original + 1 para SOS_TOKEN). O max_len do decoder deve ser max_len+1
    return torch.cat([target_with_sos, torch.tensor([vocab[PAD_TOKEN]] * (max_len + 1 - len(target_with_sos)))], dim=0)

# Gerar pares de sequências, agora com SOS no alvo
pairs = []
for _ in range(50000):
    s = random_seq(n=max_len)
    encoded_s = encode(s)
    encoded_reversed_s = encode(s[::-1])
    pairs.append((pad_input(encoded_s), pad_target(encoded_reversed_s)))


inputs = torch.stack([x for x, _ in pairs])  # Aplica padding em todas as entradas
targets = torch.stack([y for _, y in pairs])  # Aplica padding em todos os alvos

train_ds = torch.utils.data.TensorDataset(inputs, targets)  # Cria dataset PyTorch com entradas e alvos
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=128, shuffle=True)  # Cria DataLoader com batch de 128

device = 'cuda' if torch.cuda.is_available() else 'cpu'  # Seleciona GPU se disponível

# ===== 3. Prints para inspecionar =====
print(f"Vocabulário: {vocab}")  # Mostra o dicionário de tokens
print(f"Tamanho do vocabulário: {vocab_size}")  # Mostra quantos tokens existem
print(f"Tamanho máximo das sequências de entrada (max_len): {max_len}")  # Mostra o comprimento máximo
print(f"Tamanho máximo das sequências de saída (max_len + 1 para SOS): {max_len + 1}")

# Mostrar 3 exemplos codificados/decodificados
for i in range(3):
    s = random_seq()  # Gera nova sequência aleatória
    encoded = encode(s)  # Codifica para índices
    decoded = decode(encoded)  # Decodifica de volta
    reversed_s = s[::-1]
    encoded_target = encode(reversed_s)
    decoded_target = decode(pad_target(encoded_target))
    print(f"\nExemplo {i+1}:")
    print(f"  Original: {s}")
    print(f"  Codificado: {encoded.tolist()}")
    print(f"  Decodificado (entrada): {decoded}")
    print(f"  Reverso (target esperado, decodificado): {decoded_target}")

# Mostrar formas (shapes) dos tensores de entrada e saída
print("\nShapes:")
print(f"  inputs:  {inputs.shape}")  # Dimensão das entradas
print(f"  targets: {targets.shape}")  # Dimensão dos alvos

# Mostrar o primeiro batch do DataLoader
for xb, yb in train_dl:
    print("\nPrimeiro batch de treino:")
    print("  Entradas (xb):", xb.shape)  # Mostra tamanho do batch
    print("  Alvos (yb):", yb.shape)  # Mostra tamanho dos alvos
    print("  Exemplo de entrada decodificada:", decode(xb[0]))  # Converte o primeiro exemplo do batch em string
    print("  Exemplo de alvo decodificado:", decode(yb[0]))  # Converte o alvo correspondente
    break  # Mostra apenas o primeiro batch


Vocabulário: {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, ' ': 5}
Tamanho do vocabulário: 6
Tamanho máximo das sequências (max_len): 5

Exemplo 1:
  Original: deeec
  Codificado: [3, 4, 4, 4, 2]
  Decodificado: deeec
  Reverso (target esperado): ceeed

Exemplo 2:
  Original: cebcc
  Codificado: [2, 4, 1, 2, 2]
  Decodificado: cebcc
  Reverso (target esperado): ccbec

Exemplo 3:
  Original: ceaea
  Codificado: [2, 4, 0, 4, 0]
  Decodificado: ceaea
  Reverso (target esperado): aeaec

Shapes:
  inputs:  torch.Size([50000, 5])
  targets: torch.Size([50000, 5])

Primeiro batch de treino:
  Entradas (xb): torch.Size([128, 5])
  Alvos (yb): torch.Size([128, 5])
  Exemplo de entrada decodificada: eebde
  Exemplo de alvo decodificado: edbee

Excelente — esse output mostra que você **compreendeu e reproduziu um pipeline completo de preparação de dados para Machine Learning**, em um problema de **sequência para sequência (seq2seq)** simples.
Vamos analisar cada parte do resultado com foco em como isso se relaciona com o aprendizado de máquina 👇

---

## 🧩 1️⃣ Vocabulário e codificação

```
Vocabulário: {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, ' ': 5}
Tamanho do vocabulário: 6
```

👉 Isso mostra que você criou um **espaço discreto de símbolos** — uma espécie de *mini universo linguístico*.
Cada caractere foi convertido em um **índice numérico único**, e esse processo é equivalente ao que acontece em NLP (Natural Language Processing), quando as palavras são transformadas em IDs antes de virar embeddings.

**Interpretação ML:**

* Este é o **pré-processamento** de um modelo de linguagem.
* No lugar de palavras, aqui temos letras.
* Esse mapeamento (`char → int`) é a base para permitir que redes neurais operem sobre **números** em vez de texto.

---

## 🧠 2️⃣ Dados de entrada e saída — um problema *seq2seq*

```
Exemplo 1:
  Original: acdba
  Codificado: [0, 2, 3, 1, 0]
  Decodificado: acdba
  Reverso (target esperado): abdca
```

Aqui você definiu um **problema supervisionado** clássico:

* Entrada: uma sequência de letras (`acdba`)
* Saída esperada (target): a **sequência invertida** (`abdca`)

✅ Isso é um *toy problem* (problema de brinquedo) que ajuda a testar se uma rede neural consegue **aprender padrões de sequência**.
Em vez de traduzir entre idiomas (como o Encoder–Decoder faz em tradução), aqui ela precisa **aprender a inverter a sequência** — uma tarefa simples, mas perfeita para estudar aprendizado seqüencial.

---

## 🔢 3️⃣ Estruturas dos tensores

```
inputs:  torch.Size([50000, 5])
targets: torch.Size([50000, 5])
```

Isso quer dizer:

* Temos **50.000 exemplos de treino**.
* Cada exemplo é uma **sequência de 5 tokens**.

📊 Cada linha é um exemplo (amostra) e cada coluna é uma posição da sequência (caractere).
Portanto, o modelo verá isso como uma **matriz de tamanho 50.000 × 5** — um *dataset tabular temporal*.

**Do ponto de vista de ML:**

* Cada linha = uma observação.
* Cada coluna = uma dimensão temporal (ou “posição” no texto).
* Isso está no formato ideal para entrar em uma rede neural do tipo **RNN**, **LSTM** ou **Transformer Encoder–Decoder**.

---

## 🧮 4️⃣ Batch de treinamento

```
Entradas (xb): torch.Size([128, 5])
Alvos (yb): torch.Size([128, 5])
```

O **DataLoader** está dividindo o dataset em *batches* de 128 exemplos.
Isso é essencial para **treinamento eficiente** e **cálculo vetorizado em GPU**.

**Por que isso é importante:**

* As redes neurais aprendem com *gradientes médios por lote*, não amostra a amostra.
* Isso acelera o treino e suaviza o processo de otimização (SGD, Adam, etc.).

---

## 🧩 5️⃣ Conferência de um batch real

```
Exemplo de entrada decodificada: caebd
Exemplo de alvo decodificado: dbeac
```

Aqui você confirmou que:

* A **entrada** é uma sequência aleatória.
* O **alvo** é essa sequência invertida.
  Isso mostra que o dataset está **coerente e limpo**, pronto para o modelo aprender o mapeamento.

---

## 🧠 6️⃣ Interpretação conceitual (visão de ML)

| Etapa                | Conceito ML                        | Analogia                                   |
| -------------------- | ---------------------------------- | ------------------------------------------ |
| Codificação          | Transformar símbolos em números    | Dicionário de tokens                       |
| Padding              | Normalizar tamanho das sequências  | Preencher com “espaço”                     |
| Dataset + Dataloader | Estrutura de treino supervisionado | Como “perguntas e respostas” para o modelo |
| Input/Target         | Aprendizado seq2seq                | Entrada → Saída esperada                   |
| Batch                | Otimização por gradiente           | Treino em mini-grupos                      |

---

## 🎯 Conclusão

✅ **O dataset está bem construído.**
Você implementou, sem usar bibliotecas externas, o pipeline completo que qualquer sistema de NLP moderno (inclusive LLMs) usa em escala — apenas de forma reduzida e didática.

🚀 **Próximo passo natural:**

* Criar um modelo simples (por exemplo, `nn.Embedding + nn.LSTM + nn.Linear`)
* Treiná-lo para aprender a tarefa de reversão (seq2seq)
* Observar se o *loss* diminui e se o modelo aprende a gerar a sequência invertida.

Se quiser, posso gerar esse modelo de rede neural (Encoder–Decoder mínimo em PyTorch) para continuar o experimento. Quer seguir para isso?


## Veja um par

In [ ]:
print(pairs[1])

(tensor([3, 0, 3, 3, 2]), tensor([2, 3, 3, 0, 3]))


# Definição do modelo Seq2Seq com GRU

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.gru = nn.GRU(emb_size, hidden_size, batch_first=True)

    def forward(self, x):
        x = self.embed(x)
        _, h = self.gru(x) # h will be [1, B, H]
        return h

class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.gru = nn.GRU(emb_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h): # x is single token or sequence of tokens for teacher forcing
        # x is [B, 1] for single token prediction, [B, SeqLen] for teacher forcing
        x = self.embed(x)
        # out is [B, SeqLen, H] if batch_first=True
        # h is [1, B, H] after gru
        out, h = self.gru(x, h)
        logits = self.fc(out) # logits is [B, SeqLen, VocabSize]
        return logits, h # retorna o estado latente para atualizar o estado

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, tgt): # src: input sequence, tgt: target sequence (with SOS at beginning)
        h = self.encoder(src) # h from encoder is [1, B, H]
        # During training, we use teacher forcing. The decoder input is the target sequence shifted right (SOS to second to last token).
        # The decoder predicts the original target sequence (first to last token).
        logits, _ = self.decoder(tgt[:, :-1], h) # Decoder input: [B, max_len] (SOS + actual sequence up to second last)
        # logits will be [B, max_len, vocab_size], predicting for target tokens at positions 1 to max_len
        return logits


# Código para usar o modelo treinado: inferência

---



In [ ]:
def decode_step(decoder, token, h):
    # token is [B, 1] containing the last predicted token or SOS
    logits, h = decoder(token, h) # logits: [B, 1, VocabSize], h: [1, B, H]
    next_token = logits.argmax(-1) # next_token: [B, 1]
    return next_token, h

def predict(model, seq, max_len_output):
    model.eval()
    with torch.no_grad():
        # Encoder input (src) needs to be padded to max_len (5)
        src_encoded = encode(seq) # encode the input string
        # Pad the encoded source to max_len
        src = pad_input(src_encoded).unsqueeze(0).to(device, dtype=torch.long) # src: [1, max_len]

        h = model.encoder(src) # h: [1, 1, hidden_size]

        # Start decoding with the SOS_TOKEN
        token = torch.tensor([[vocab[SOS_TOKEN]]], dtype=torch.long, device=device) # token: [1, 1]

        seq_invertida_tokens = []
        # max_len_output is the length of the original string (e.g., 5)
        # The loop runs for max_len_output steps to generate max_len_output characters
        for _ in range(max_len_output):
            token, h = decode_step(model.decoder, token, h) # token and h are updated
            seq_invertida_tokens.append(token.item())

            # Stop decoding if PAD_TOKEN is predicted (or any other stop condition)
            if token.item() == vocab[PAD_TOKEN]:
                break

        # Decode the generated tokens, ignoring SOS/PAD during final string assembly
        return ''.join(inv_vocab[t] for t in seq_invertida_tokens if inv_vocab[t] not in [PAD_TOKEN, SOS_TOKEN])


# Preparação para treino

In [ ]:
emb_size = 32
hidden_size = 64
# The vocab_size includes all chars + PAD_TOKEN + SOS_TOKEN
encoder = Encoder(vocab_size, emb_size, hidden_size)
decoder = Decoder(vocab_size, emb_size, hidden_size)
model = Seq2Seq(encoder, decoder).to(device)

# Ignore PAD_TOKEN in loss calculation
# The target for loss is yb[:, 1:] because yb[:, :-1] is the input to the decoder
# So yb[:, 1:] represents the tokens that the decoder should predict.
loss_fn = nn.CrossEntropyLoss(ignore_index=vocab[PAD_TOKEN])
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

# Execução do treino

In [ ]:
epochs = 20 # Can increase if needed

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for xb, yb in train_dl:
        xb, yb = xb.to(device, dtype=torch.long), yb.to(device, dtype=torch.long)
        opt.zero_grad()
        # logits will have shape [B, max_len_target_seq, vocab_size] (where max_len_target_seq = max_len_original + 1 for SOS)
        logits = model(xb, yb) # The model's forward pass now correctly handles teacher forcing with SOS

        # Target for loss is yb[:, 1:] because yb[:, :-1] is fed to decoder
        # yb[:, 1:] has shape [B, max_len_output], so reshape to [B * max_len_output]
        # logits has shape [B, max_len_output, vocab_size], reshape to [B * max_len_output, vocab_size]
        loss = loss_fn(logits.reshape(-1, vocab_size), yb[:, 1:].reshape(-1)) # yb[:, 1:] is actual target values, excluding the initial SOS
        loss.backward()
        opt.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: loss={total_loss/len(train_dl):.4f}")

print("Treino concluído.")

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.gru = nn.GRU(emb_size, hidden_size, batch_first=True)

    def forward(self, x):
        x = self.embed(x)
        _, h = self.gru(x)
        return h  # [1, B, H]

class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.gru = nn.GRU(emb_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h):
        """
        x: tensor que indica a parte prévia correta
        h: tensor que indica o estado do encoder da parte prévia
        """
        x = self.embed(x)
        out, h = self.gru(x, h)
        logits = self.fc(out)
        return logits, h # retorna o estado latente para atualizar o estado

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, tgt):
        h = self.encoder(src)
        # usa contexto correto anterior e estado atual para prever o tgt[:, -1]
        logits, _ = self.decoder(tgt[:, :-1], h)
        return logits

In [ ]:
emb_size = 32
hidden_size = 64
encoder = Encoder(vocab_size, emb_size, hidden_size)
decoder = Decoder(vocab_size, emb_size, hidden_size)
model = Seq2Seq(encoder, decoder).to(device)

loss_fn = nn.CrossEntropyLoss(ignore_index=vocab[' ']) # ignora o pad: " "
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

print(f"Model initialized and moved to device: {device}")
print(f"Model architecture:\n{model}")

# Vamos testar

## Avaliação da acurácia do modelo

In [ ]:
def calculate_accuracy(model, dataset, decode_fn, vocab, max_len):
    correct_predictions = 0
    total_predictions = 0

    model.eval()
    with torch.no_grad():
        for src_tensor, tgt_tensor in dataset:
            src_str_decoded = decode_fn(src_tensor.cpu()) # Decode input string without PAD/SOS

            # Expected target is the original string reversed. decode_fn for target ignores SOS/PAD.
            expected_str = decode_fn(tgt_tensor.cpu()) # This will automatically strip SOS/PAD for target

            # The predict function now returns the string without PAD/SOS
            predicted_str = predict(model, src_str_decoded, max_len_output=max_len)

            if predicted_str == expected_str:
                correct_predictions += 1
            total_predictions += 1

    return correct_predictions / total_predictions

# Calculate accuracy on the training dataset
accuracy = calculate_accuracy(model, train_ds, decode, vocab, max_len)
print(f"Acurácia do modelo no dataset de treino: {accuracy:.4f}")

## Exemplos de Previsão do Modelo

Vamos observar algumas previsões do modelo e compará-las com o resultado esperado.

In [ ]:
correct_examples = []
incorrect_examples = []

# max_len is the length of the actual string (e.g., 5)
for _ in range(20): # Test with 20 random sequences
    s = random_seq(n=max_len) # Generate sequence of max_len
    expected_reversed = s[::-1] # True reversed sequence
    # predict function now takes max_len_output which is the length of the original string
    pred = predict(model, s, max_len_output=max_len)

    if pred == expected_reversed:
        correct_examples.append((s, pred, expected_reversed))
    else:
        incorrect_examples.append((s, pred, expected_reversed))

print("\n--- Exemplos Corretos ---")
for i, (original, predicted, expected) in enumerate(correct_examples):
    if i >= 5: break # Limit to 5 examples
    print(f"Original: '{original}' -> Previsto: '{predicted}' (Esperado: '{expected}')")

print("\n--- Exemplos Incorretos ---")
for i, (original, predicted, expected) in enumerate(incorrect_examples):
    if i >= 5: break # Limit to 5 examples
    print(f"Original: '{original}' -> Previsto: '{predicted}' (Esperado: '{expected}')")

if not incorrect_examples:
    print("Não foram encontrados exemplos incorretos entre as amostras testadas.")
else:
    print(f"Total de exemplos corretos: {len(correct_examples)}")
    print(f"Total de exemplos incorretos: {len(incorrect_examples)}")

In [ ]:
print("\n--- Testes de previsão avulsos ---")
for _ in range(10):
    s = random_seq(n=max_len) # Use max_len for consistency
    print(f"Input string: '{s}'") # Added print to show the generated input
    # Use max_len_output=max_len
    pred = predict(model, s, max_len_output=max_len)
    print(f"{s} -> {pred} (Esperado: {s[::-1]})")